## The Schema

In [1]:
from typing import Optional

from pydantic import BaseModel, Field

class Person(BaseModel):
  """Information about a person."""
  name: Optional[str] = Field(
    default=None, 
    description="The name of the person"
  )
  hair_color: Optional[str] = Field(
    default=None,
    description="The color of the person's hair if known"
  )
  height_in_meters: Optional[str] = Field(
    default=None, description="Height measured in meters"
  )

## The Extractor

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
  [
    ("system",
     "You are an expert extraction algorithm. "
     "Only extract relevant information from the text. "
     "If you do not know the value of an attribute asked to extract, "
     "return null for the attribute's value."),
    ("human",
     "{text}")
  ]
)

In [4]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [5]:
structured_llm = llm.with_structured_output(Person)

In [6]:
text = "Dat is 6 ft tall and has black hair"
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='Dat', hair_color='black', height_in_meters='1.83')

In [7]:
from typing import Optional, List
from pydantic import BaseModel, Field

class Person(BaseModel):
  """Information about a person."""
  name: Optional[str] = Field(
    default=None,
    description="The name of a person"
  )
  hair_color: Optional[str] = Field(
    default=None,
    description="The color of the person's hair if known"
  )
  height_in_meters: Optional[str] = Field(
    default=None, description="Height measured in meters"
  )
  
class Data(BaseModel):
  """Extracted data about people."""
  people: List[Person]

In [8]:
structured_llm = llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Data(people=[Person(name='Jeff', hair_color='black', height_in_meters='1.83'), Person(name='Anna', hair_color='black', height_in_meters=None)])

In [9]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content)

7


In [10]:
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
  (
    "The ocean is vast and blue. It's more than 20,000 feet deep.",
    Data(people=[])
  ),
  (
    "Fiona traveled far from France to Spain.",
    Data(people=[Person(name="Fiona", height_in_meters=None, hair_color=None)]),
  )
]

messages = []

for txt, tool_call in examples:
  if tool_call.people:
    ai_response = "Detected people."
  else:
    ai_response = "Detected no people."
    
  messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

/var/folders/y6/f01_tq9n4sv37v7r4hpwxq840000gn/T/ipykernel_42504/734817787.py:22: LangChainBetaWarning: The function `tool_example_to_messages` is in beta. It is actively being worked on, so the API may change.
  messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))


In [13]:
for mess in messages:
  mess.pretty_print()

================================ Human Message =================================

The ocean is vast and blue. It's more than 20,000 feet deep.
================================== Ai Message ==================================
Tool Calls:
  Data (55a944be-475e-449d-b69d-0ab075daef5d)
 Call ID: 55a944be-475e-449d-b69d-0ab075daef5d
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected no people.
================================ Human Message =================================

Fiona traveled far from France to Spain.
================================== Ai Message ==================================
Tool Calls:
  Data (05648183-0ddb-4ab4-92e5-adc0a59956a9)
 Call ID: 05648183-0ddb-4ab4-92e5-adc0a59956a9
  Args:
    people: [{'name': 'Fiona', 'hair_color': None, 'height_in_meters': None}]
================================

In [15]:
message_no_extraction = {
  "role": "user",
  "content": "The solar system is large, but earth has only 1 moon."
}

structured_llm = llm.with_structured_output(schema=Data)
structured_llm.invoke([message_no_extraction])

Data(people=[Person(name='earth', hair_color=None, height_in_meters=None)])

In [16]:
structured_llm.invoke(messages + [message_no_extraction])

Data(people=[])